<br>**Заказчик исследования:** инвесторы фонда «Shut Up and Take My Money», планирующие открыть заведение общественного питания в Москве
<br>**Цель исследования:** подготовить исследование рынка Москвы, найти интересные особенности, которые помогут в выборе подходящей локации
<br>**База для исследования:** датасет с заведениями общественного питания Москвы, составленный на основе данных сервисов Яндекс Карты и Яндекс Бизнес на лето 2022 года
<br>
<br>
Для достижения поставленных целей необходимо выполнить следующие этапы исследования:
<br>
1. Провести подготовку полученных данных
2. Проанализировать данные по рынку в целом
3. Проанализовать данные по сегменту кофеен, предложить потенциально успешную локацию для нового заведения
4. Подготовить презентацию с основными выводами и рекомендациями

# 1. Подготовка и предобработка данных

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import plotly.express as px
import folium
from folium import Map, Choropleth
import json

In [ ]:
try:
    data = pd.read_csv('moscow_places.csv') # загружаем локальный файл с данными
except:
    data = pd.read_csv('/datasets/moscow_places.csv')   # альтернативный пусть к данным для работы с других устройств

In [ ]:
plt.style.use('seaborn-colorblind') # устанавливаем единиый стиль для визуализации с помощью библиотеки seaborn

Запустим функцию для первичного анализа данных. Проанализируем есть ли пропуски и явные дубликаты:

In [ ]:
def initial_analysis(file):  
    print('Общая информация о датасете:')
    display(file.info())
    print ('-----------------------------')
    print('Первые 5 строчек датасета:')
    display(file.head(5))
    print ('-----------------------------')
    print('Количество пропусков в датасете:')    
    display(data.isna().sum())
    print ('-----------------------------')
    print('Количество явных дубликатов в датасете:')    
    display(data.duplicated().sum())

In [ ]:
initial_analysis(data)

Представленный датасет содержит информацию о 8406 точках общественного питания. Приведенные названия столбцов корректны. Явные дубликаты в базе отсутствуют. 
Пропуски отмечаются в полях средний чек, средняя стоимость чашки кофе, категория цен в заведении, количество посадочных мест и часы работы.
В столбце chain представляется желательным изменить тип данных на булевый, т.к возможны значения только 0 и 1.

Необходимо проверить датасет на наличие неявных дубликатов, а также добавить новые столбцы: street с названиями улиц из столбца с адресом.
и is_24/7 с обозначением, что заведение работает ежедневно и круглосуточно.

In [ ]:
data['chain']= data['chain'].astype(bool)  # переведем тип данных в столбце chain в bool

In [ ]:
 data.info()

Приведем все названия заведений к нижнему регистру и посмотрим появились ли дубликаты теперь:

In [ ]:
data['name'] = data['name'].str.lower()
data.duplicated().sum()

Дубликатов по имени не появилось, посмотрим есть ли неявные дубликаты в сочетании название заведения + адрес 

In [ ]:
data[["name", "address"]].duplicated().sum()

Найдем их в датасете:

In [ ]:
data.loc[data[["name", "address"]].duplicated()]

Рассмотрим каждое из этих заведений детально:

In [ ]:
data.query('name == "more poke"')

Перед нами явный дубликат - совпадают все данные кроме часов работы и указания сетевое/нет заведение. Оставляем вариант с сетевым - т.к. есть еще один ресторан в Духовском переулке.  

In [ ]:
data.query('name == "хлеб да выпечка"')

Дубликат, задвоение произошло видимо из-за разного названия категорий. Оставляем вариант с "булочной" - оно соответствует концепции заведения

In [ ]:
data.query('name == "раковарня клешни и хвосты"')

По одному адресу и с одним названием работает бар и ресторан, причем бар работает дольше чем ресторан по выходным. Выглядит достаточно реалистично, не считаем дубликатом. 

Первые два варианта дубликата удаляем из датасета.

In [ ]:
list_index = [1430, 3109]
data = data.query('index not in @list_index')

In [ ]:
data.info()

Удалилось два заведения. Переходим к выделению улиц из адресов заведений, используем метод split

In [ ]:
data['street'] = data['address'].str.split(',', expand = True)[1]

In [ ]:
data.head()

Выделив улицы в отдельный столбец для дальнейшего анализа переходим к созданию столбца с информацией круглосуточно ли и всю неделю работает заведение:

In [ ]:
data['is_24/7'] = "" # создаем пустой столбец

In [ ]:
# заполняем его значениями True в случае если в столбце hours указано, что заведение работает "ежедневно, круглосуточно"

data.loc[data['hours'] == 'ежедневно, круглосуточно', 'is_24/7'] = 'True'  

In [ ]:
# u False во всех других ситуациях

data.loc[data['is_24/7'] == '', 'is_24/7'] = 'False'

In [ ]:
data.head(20)

В завершении подготовки данных проанализируем как обстоит дело с выбросами в столбцах, содержащих числовые даные:

In [ ]:
data.describe()

Внешний вид разброса данных с географической широтой и долготой выглядит корректно, как и рейтинг - значения от 1 до 5, среднее 4.2
<br>
<br>
Обращают на себя внимания наличие выбросов в полях middle_avg_bill, middle_coffee_cup и seats. Посмотрим на них внимательнее:

In [ ]:
data.query('middle_avg_bill < 50') 

Средний чек в заведении с шаурмой может быть 30 рублей, а вот в кофемании - нет. Уберем это значение из датасета.

In [ ]:
# меняем значение на пустое

data.loc[[3688], 'middle_avg_bill'] = data.loc[[3688], 'middle_avg_bill'].replace(0.0, np.nan) 

Теперь посмотрим на заведения со средним чеком выше 15000 

In [ ]:
data.query('middle_avg_bill > 15000')

Возможно речь идет о банкетном зале, который ориентирован на большие мероприятия (свадьбы и тп), тогда средний чек возможен. Оставляем.

Посмотрим на выброс в стоимость 1 чашки кофе. 

In [ ]:
data.query('middle_coffee_cup > 500')

В данных по шоколаднице ошибка, заполним значение средним по сети:

In [ ]:
data.loc[[2859], 'middle_coffee_cup']  = data.query('name == "шоколадница"')['middle_coffee_cup'].median()
data.describe()

Осталось разобраться с посадочными местами.  Посмотрим на все заведения с количеством посадочных мест более 1000:

In [ ]:
(data
    .query('seats > 1000')
    .pivot_table(index = 'address', values = 'name', aggfunc= 'count')
    .reset_index()
    .sort_values('name', ascending=False)
)

Если посмотреть детально эти адреса, то по ним расположены крупные развлекательные заведения, содержащие в себе большое количество ресторанов. Сумма посадочных мест, вероятно,указывается общая. Оставляем эти данные без изменений, но учитываем при дальнейшем анализе - работаем с медианными, а не средними показателями.

**Вывод:** предобработка данных завершена. В ее результате были удалены 2 дубликата и 1 выброс, данные проанализированы на предмет их корректности, подготовлены необходимые для дальнейшей работы поля. База готова к анализу.

# 2. Анализ данных

## 2.1. Категории заведений 

 Посмотрим какие виды заведений представлены в нашей базе:

In [ ]:
data['category'].unique()

In [ ]:
category =(data
             .groupby('category')  # группируем по категориям
            .agg(count=('address','count')) # считаем количество адресов
           .reset_index()
           .sort_values('count', ascending=False)  # сортируем по убыванию
)
category['percent'] = round((category['count'] / sum(category['count']) * 100), 2)  # добавляем рассчет доли
category

In [ ]:
plt.figure(figsize=(10, 3))
ax = sns.barplot( x='category', y='count', data=category,  alpha=0.8)   # строим барплот на основе рассчитанных данных
ax.set_title('Количество заведений по категориям')
ax.set_xlabel('Категория заведений') 
plt.xticks(rotation = 25)
ax.set_ylabel('Количество')
plt.show()

Всего по базе данных на лето 2022 года в Москве 8004 заведений общественного питания. Среди них больше всего заведений с общей категорией кафе (28% ), за ними следуют рестораны (24%) и кофейни (17%). 

Посмотрим на количество посадочных мест по категориям:

In [ ]:
seats =(data
        .groupby('category')
        .agg(median=('seats','median'))
        .reset_index()
        .sort_values('median', ascending=False)
)
seats

In [ ]:
plt.figure(figsize=(10, 3))
ax = sns.barplot(x='category', y='median',  data=seats,  alpha=0.8) 
ax.set_title('Медианное количество посадочных мест по категориям заведений')
ax.set_xlabel('Категория заведений') 
plt.xticks(rotation = 25)
ax.set_ylabel('Количество мест')
plt.show()

Больше всего посадочных мест в ресторанах, барах и кофейнях. У чаще всего встречающихся кафе только 6 место, меньше всего мест обычно в булочных.

## 2.2. Сетевые и несетевые заведения на рынке общественного питания Москвы 

Посмотрим количество сетевых и несетевых заведений на рынке Москвы:

In [ ]:
chain_all = (data
         .pivot_table(index = 'chain', values = 'address', aggfunc = 'count' )
         .reset_index()
)  # создадим сводную таблицу
chain_all
chain_all.columns = ['Chain/not', 'qnt' ]
chain_all['chain_percent'] = round(chain_all['qnt']/sum(chain_all['qnt'])*100,2)  # рассчитаем процентное соотношение
chain_all = chain_all.sort_values('chain_percent', ascending=False)
chain_all

Нарисуем pie chart:

In [ ]:
plt.figure(figsize=(8, 4))
labels = ['несетевое', 'сетевое'] # устанавливаем подписи 
colors = sns.color_palette('pastel')[0:5]  # цветовую палитру
# создаем диаграмму, подписывая значения:
plt.pie(chain_all['chain_percent'], labels = labels, colors=colors, autopct='%.0f%%')  
plt.title('Соотношение сетевых и несетевых заведений на рынке Москвы')
plt.show()


Посмотрим аналогичное соотношение в разрезе по категориям заведений:

In [ ]:
chain = (data
         .pivot_table(index = 'category', columns = 'chain', values = 'address', aggfunc = 'count' )
         .reset_index()
)
chain.columns = ['category', 'not_chain', 'chain' ]
chain['chain_percent'] = round(chain['chain']/(chain['chain']+chain['not_chain'])*100,2)
chain = chain.sort_values('chain_percent', ascending=False)
chain

In [ ]:
plt.figure(figsize=(10, 3))
sns.barplot(data=chain, x='category',y='chain_percent',  alpha=0.8)
plt.title('Доля сетевых заведений по категориям')
plt.xlabel('Категория заведений')
plt.xticks(rotation = 25)
plt.ylabel('Процентная доля сетевых игроков')
plt.show()

Среди булочных, пиццерий и кофеен сетевых игроков больше половины, что заметно выше среднего значения по рынку 38%. Больше всего сетевых заведений среди булочных - 61%

## 2.3. ТОП-15 популярных сетей 

Сгруппируем данные по названиям заведений и найдем топ-15 популярных сетей в Москве с наибольшим количеством заведений

In [ ]:
top_15 = (data
          .query('chain == True')  # выбираем сетевые заведения
          .groupby('name')  # группируем по имени
          .agg(category=('category', 'unique'),count=('address','count'))  # добавляем категории
          .reset_index()
          .sort_values('count', ascending=False) # сортируем по уменьшению количества
          .head(15)  # отбираем первые 15
)
top_15

In [ ]:
top_15.loc[[418], 'name'] = 'кулинар. лавка\n караваевых'  # видоизменим длинное название для визуализации данных

In [ ]:
plt.figure(figsize=(10, 3))
sns.barplot(data=top_15, x='name',y='count',  alpha=0.8)
plt.title('ТОП-15 самых популярных сетевых заведений')
plt.xlabel('Название сети')
plt.ylabel('Количество заведений в сети')
plt.xticks(rotation = 45) # поворачиваем подписи по оси X для удобного прочтения графика
plt.show()

Больше всего в Москве заведений сети "Шоколадница", "Доминос'c пицца", "Додо пицца" и "One price coffee" - таким образом в лидерах две пиццерии, и две кофейни. В ТОП-15 находятся преимущественно заведения быстрого питания, пиццерии или кофейни. 

# 2.4. Разрез рынка по административным районам Москвы


Посмотрим какие административные районы Москвы присутствуют в датасете, отобразим количество заведений и количество заведений каждой категории по районам

In [ ]:
data['district'].unique() 

In [ ]:
district_all = (data
          .groupby('district')  # группируем по округу
          .agg(count=('address','count'))  # считаем количество адресов
          .reset_index()
          .sort_values('count', ascending=False) 
        )

# используем сокращения названия районов для упрощения визуализации:
district_all['district'] = district_all['district'].replace(
    ['Центральный административный округ', 'Северный административный округ', \
     'Южный административный округ', 'Северо-Восточный административный округ', 'Западный административный округ', \
     'Восточный административный округ', 'Юго-Восточный административный округ', 'Юго-Западный административный округ', \
     'Северо-Западный административный округ'], 
    ['ЦАО', 'САО', 'ЮАО', 'СВАО', 'ЗАО', 'ВАО', 'ЮВАО', 'ЮЗАО', 'СЗАО'])
             
district_all     

In [ ]:
plt.figure(figsize=(8, 3))
sns.barplot(x='district', y = 'count', data=district_all)
plt.title('Количество заведений по административным округам')
plt.xlabel('Административный округ')
plt.ylabel('Количество заведений')
plt.grid() # добавляем сетку
plt.show()

Больше всего заведений в ЦАО (2242), распределение заведений по остальным районам выглядит достаточно равномерно (от 700 до 900) за исключением СЗАО - там их меньше 500.

In [ ]:
# сгруппируем данные по районам и категориями

district = data.groupby(['district', 'category'], as_index = False)[['name']]\
            .count().sort_values('name', ascending = False)

# используем сокращения названия районов для упрощения визуализации:
district['district'] = district['district'].replace(
    ['Центральный административный округ', 'Северный административный округ', \
     'Южный административный округ', 'Северо-Восточный административный округ', 'Западный административный округ', \
     'Восточный административный округ', 'Юго-Восточный административный округ', 'Юго-Западный административный округ', \
     'Северо-Западный административный округ'], 
    ['ЦАО', 'САО', 'ЮАО', 'СВАО', 'ЗАО', 'ВАО', 'ЮВАО', 'ЮЗАО', 'СЗАО'])

Визуализируем с помощью столбчатого графика средствами seaborn:

In [ ]:
plt.figure(figsize=(10, 5))
sns.barplot(x='district', y = 'name', data=district, hue='category')
plt.title('Количество заведений каждой категории по административным округам')
plt.xlabel('Административный округ')
plt.ylabel('Количество заведений')
plt.legend(loc='upper right', fontsize=8) # выбираем положение легенды и указываем размер шрифта
plt.grid() # добавляем сетку
plt.show()

В разных районах свои лидеры - в ЦАО больше всего ресторанов, в остальных районах - кафе. 

## 2.5. Рейтинг заведений по категориям


Визуализируем распределение средних рейтингов по категориям заведений и посмотрим сильно ли различаются усреднённые рейтинги в разных типах общепита

In [ ]:
round(data['rating'].mean(),1)

Из проведенного в первой части анализа базы мы помним, что в рейтингах отсутствуют выбросы, поэтому рассчитываем общее среднее - 4.2 балла - средняя оценка по всем заведениям рынка Москвы. Посмотрим детальнее по категориям:

In [ ]:
rating = round(data.groupby('category')['rating'].mean(), 1).sort_values(ascending = False).reset_index()
rating

In [ ]:
plt.figure(figsize=(8, 3))
ax = sns.barplot(data=rating, y='rating', x='category', alpha=0.8)
ax.set_title('Средний рейтинг по категориям')
ax.set_xlabel('Средний рейтинг')
ax.set_ylabel('Категории заведений')
plt.xticks(rotation = 25)
ax.set_ylim((3, 5))   # т.к. разница в рейтингах незначительна, то установим ограничение по оси y для лучшей визуализации

plt.show()

Самую высокую оценку получили бары - 4.4, выше среднего также были оценены булочные, кофейни, пиццерии и рестораны (4.3). А вот рестораны быстрого питания и кафе чаще получают оценку ниже среднего - 4.1. 

## 2.6. Рейтинг заведений в разрезе по административным округам Москвы

Построим фоновую картограмму  со средним рейтингом заведений каждого района

In [ ]:
district_rating = round(data.groupby('district')['rating'].mean(), 1).sort_values(ascending = False).reset_index()
district_rating

In [ ]:
state_geo = 'https://code.s3.yandex.net/data-analyst/admin_level_geomap.geojson'  # границы адмнистративных округов

In [ ]:
# moscow_lat - широта, moscow_lng – долгота
moscow_lat, moscow_lng = 55.751244, 37.618423
# создаём объект m – карту с центром в точке с координатами [moscow_lat, moscow_lng]
moscow_rating = folium.Map(location=[moscow_lat, moscow_lng], zoom_start=10)

In [ ]:
# создаём хороплет с помощью конструктора Choropleth и добавляем его на карту
Choropleth(
    geo_data=state_geo,
    data=district_rating,
    columns=['district', 'rating'],
    key_on='feature.name',
    fill_color='YlGn',
    fill_opacity=0.8,
    legend_name='Средний рейтинг заведений по районам',
).add_to(moscow_rating)

# выводим карту
moscow_rating

Самый высокий рейтинг - у заведений в ЦАО, ниже всего - в ЮВАО и САО.

 Теперь отобразим все заведения датасета на карте с помощью кластеров средствами библиотеки folium.

In [ ]:
# импортируем карту и маркер
from folium import Map, Marker
# импортируем кластер
from folium.plugins import MarkerCluster

# создаём пустой кластер, добавляем его на карту
marker_cluster = MarkerCluster().add_to(moscow_rating)

# пишем функцию, которая принимает строку датафрейма,
# создаёт маркер в текущей точке и добавляет его в кластер marker_cluster
def create_clusters(row):
    Marker(
        [row['lat'], row['lng']],
        popup=f"{row['name']} {row['rating']}",
    ).add_to(marker_cluster)

# применяем функцию create_clusters() к каждой строке датафрейма
data.apply(create_clusters, axis=1)

# выводим карту
moscow_rating

ЦАО - несомненный лидер по количеству заведений, остальные заведения достаточно равномерно запределены по районам города, кроме юго-востока - там их меньше.

## 2.7. ТОП-15 улиц Москвы по количеству заведений


Найдем топ-15 улиц по количеству заведений. Построим график распределения количества заведений и их категорий по этим улицам. 

In [ ]:
top_15_street = (data
                 .groupby('street')
                 .agg(qnt=('address','count')).reset_index()
                 .sort_values('qnt', ascending=False)
                 .head(15)
                )
top_15_street 

In [ ]:
top_15_street_list = top_15_street['street'].to_list()  # передаем список улиц в новый лист
top_15_street_list

In [ ]:
top_street_category = (data
                        .query('street in @top_15_street_list')  # отбираем из датасета данные по топ-15 улицам
                       # формируем сводную таблицу с уникальными названиями и категорями
                       .pivot_table(index = ['street','category'], values = 'name', aggfunc = 'nunique')
                       .reset_index()
                      )
top_street_category.columns = ['street', 'category', 'quantity']

In [ ]:
# визуализируем информацию с помощью интерактивного графика:

fig = px.bar(top_street_category,  x='quantity', y='street', color='category', \
             title='Распределение количества заведений и их категорий на топ-15 улицах')
fig.update_xaxes(title_text = 'Количество заведений')
fig.update_yaxes(title_text = '', categoryorder = 'total ascending')  # сортируем данные по убыванию общего кол-ва заведений
fig.show()

Среди ТОП-15 самых популярных по количеству заведений улиц – крупные магистрали и большие, хорошо проходимые улицы - проспекты Мира, Вернадского, Ленинский, Ленинградский, Профсоюзная улица, Дмитровское, Варшавское, Каширское, Ленинградское шоссе и т.п. - все это протяженные, крупные транспортные артерии со значительным клиентопотоком.
Проспект Мира - явный лидер по количеству заведений. Больше всего на ТОП-15 улицах ресторанов и кафе, за ними следуют кофейни, пиццерии и рестораны быстрого питания.

## 2.8. Улицы Москвы, где находится только один объект общепита


Найдем улицы, на которых находится только один объект общепита:

In [ ]:
anti_top_street = (data.groupby('street')
                   .agg(qnt=('address','nunique'))
                   .reset_index().sort_values('qnt', ascending=True)
                   .query('qnt ==1')
                  )

anti_top_street_list = anti_top_street['street'].to_list()  # сформируем лист с названиями этих улиц

Выгрузим из датасета информацию по заведениям, которые являются единственными на своих улицах:

In [ ]:
anti_top = data.query('street in @anti_top_street_list')
anti_top.groupby('category').agg(qnt = ('name', 'nunique')).reset_index().sort_values('qnt', ascending = False) 

Среди них больше всего кафе, ресторанов и кофеен. Посмотрим как они распределяются географически:

In [ ]:
moscow_anti_street = folium.Map(location=[moscow_lat, moscow_lng], zoom_start=10) # создадим новую карту Москвы

# создаём пустой кластер, добавляем его на карту
marker_cluster = MarkerCluster().add_to(moscow_anti_street)


# пишем функцию, которая принимает строку датафрейма,
# создаёт маркер в текущей точке и добавляет его в кластер marker_cluster
def create_clusters(row):
    Marker(
        [row['lat'], row['lng']],
        popup=f"{row['name']} {row['rating']}",
    ).add_to(marker_cluster)

# применяем функцию create_clusters() к каждой строке датафрейма
anti_top.apply(create_clusters, axis=1)

# выводим карту
moscow_anti_street

Проанализировав карту можно сделать вывод, что заведения, которые являются единственными на своих улицах расположены достаточно равномерно по всему городу. Улицы, на которых они расположены, преимущественно небольшие, расположены далеко от основных улиц и магистралей и предположительно обладают небольшим клиентопотоком.

## 2.9. Средний чек заведений


Посчитаем медиану среднего чека для каждого района. Используем это значение в качестве ценового индикатора района и посмотрим фоновую картограмму (хороплет) с полученными значениями для каждого района. 

In [ ]:
district_avg_prices = (data.groupby('district')
                       .agg(avg_bill=('middle_avg_bill', 'median'))
                       .reset_index()
                       .sort_values('avg_bill', ascending = False)
                      )

district_avg_prices

In [ ]:
plt.figure(figsize=(8, 2))
ax = sns.barplot(data=district_avg_prices, x='avg_bill', y='district', alpha=0.8)

ax.set_title('Средний чек по районам')
ax.set_ylabel('Административный округ')
ax.set_xlabel('Средний чек')


plt.show()

In [ ]:
# создаём объект moscow_prices – новую карту с центром в точке с координатами [moscow_lat, moscow_lng]
moscow_prices= folium.Map(location=[moscow_lat, moscow_lng], zoom_start=10)

#создаём хороплет с помощью конструктора Choropleth и добавляем его на карту
Choropleth(
    geo_data=state_geo,
    data=district_avg_prices,
    columns=['district', 'avg_bill'],
    key_on='feature.name',
    fill_color='YlGn',
    fill_opacity=0.8,
    legend_name='Средний чек заведений по районам',
).add_to(moscow_prices)

# выводим карту
moscow_prices

Самые высокие цены в ЦАО и ЗАО - средний чек составляет 1000 рублей. Можно заключить, что приближенность к центру влияет на рост среднего чека, помимо этого на него также влияет и общая престижность района - так, ЗАО хоть и удален от центра, но при этом является достаточно дорогим районом с соответствующими заведениями и уровнем цен в них.

## 2.10 Общие выводы по анализу рынка общепита Москвы

Всего по базе данных на лето 2022 года в Москве 8004 заведений общественного питания.
Среди них больше всего заведений с общей категорией кафе (28% ), за ними следуют рестораны (24%) и кофейни (17%).
<br>
<br>
**Посадочных мест** в среднем больше в ресторанах, барах и кофейнях – порядка 80-90 мест. Меньше всего мест обычно в булочных – около 50.
<br>
<br>
**Крупные сети.** В соотношении по рынку всего 38% - сетевые заведения. При этом среди булочных, пиццерий и кофеен сетевых игроков заметно выше - больше половины. Больше всего сетевых заведений среди булочных - 61%.
Больше всего в Москве заведений сети "Шоколадница", "Доминос'c пицца", "Додо пицца" и "One price coffee" - таким образом в лидерах две пиццерии, и две кофейни. Среди 15 самых популярных сетей Москвы преимущественно заведения быстрого питания, пиццерии или кофейни.
<br>
<br>
**Рейтинг.**  Средняя оценка по всем заведениям рынка Москвы 4.2 балла  из 5, что говорит о достаточно высоком среднем уровне качества заведений. Самую высокую оценку в среднем получают бары - 4.4, выше среднего также были оценены булочные, кофейни, пиццерии и рестораны (4.3). А вот рестораны быстрого питания и кафе чаще получают оценку ниже среднего - 4.1.
В разрезе по районам самый высокий рейтинг - у заведений в ЦАО, ниже всего - в ЮВАО и САО.
<br>
<br>
**Локации.** Больше всего заведений в ЦАО (2242), распределение заведений по остальным районам выглядит достаточно равномерно (от 700 до 900) за исключением СЗАО - там их меньше 500.
В разных районах наблюдаются свои лидеры по категориям заведений - например в ЦАО больше всего ресторанов, в остальных районах - кафе.
<br>
<br>
**Популярные и непопулярные улицы Москвы.**
Явный лидер по количеству заведений в Москве - Проспект Мира. Среди ТОП-15 самых популярных по количеству заведений улиц – крупные магистрали и большие, хорошо проходимые улицы с значительным клиентопотоком. Больше всего на ТОП-15 улицах ресторанов и кафе, за ними следуют кофейни, пиццерии и рестораны быстрого питания.
Непопулярные улицы (на которых всего одно заведение) расположены достаточно равномерно по всему городу, преимущественно являются небольшими, расположенными далеко от основных улиц и магистралей.
<br>
<br>
**Цены.** Самые высокие цены в ЦАО и ЗАО - средний чек составляет 1000 рублей. Можно заключить, что приближенность к центру влияет на рост среднего чека, помимо этого на него также влияет и общая престижность района - так, ЗАО хоть и удален от центра, но при этом является достаточно дорогим районом с соответствующими заведениями и уровнем цен в них.

# 3. Детализированное исследование: открытие кофейни


Проанализируем сегмент кофеен более детально. Всего среди заведений общественного питания Москвы по состоянию на лето 2022 года 1413 кофеен, что составляет 17% от их общего числа. 

## 3.1 Общее количество и особенности расположений кофеен

In [ ]:
coffee_data = data.query('category == "кофейня"')  # выделяем для анализа датасет с одними кофейнями
coffee_data.info()

Посмотрим какие особенности расположения кофеен:

In [ ]:
сoffee_district = (coffee_data.groupby('district')
     .agg(qnt=('address', 'nunique')).reset_index()
     .sort_values('qnt', ascending = False)
)
сoffee_district['percent'] = round(сoffee_district['qnt']/сoffee_district['qnt'].sum()*100, 1)
сoffee_district

Больше всего кофеен в ЦАО - свыше 30%.

Построим карту с кластерами:

In [ ]:
coffee_map= folium.Map(location=[moscow_lat, moscow_lng], zoom_start=10)

# создаём пустой кластер, добавляем его на карту
marker_cluster = MarkerCluster().add_to(coffee_map)


# пишем функцию, которая принимает строку датафрейма,
# создаёт маркер в текущей точке и добавляет его в кластер marker_cluster
def create_clusters(row):
    Marker(
        [row['lat'], row['lng']],
        popup=f"{row['name']} {row['rating']}",
    ).add_to(marker_cluster)

# применяем функцию create_clusters() к каждой строке датафрейма
coffee_data.apply(create_clusters, axis=1)

# выводим карту
coffee_map

## 3.2 Режим работы кофеен

Посмотрим есть ли круглосуточные кофейни и какой у них рейтинг:

In [ ]:
coffee_24 = coffee_data.groupby('is_24/7').agg(qnt = ('name', 'count'), rating = ('rating', 'mean'))
coffee_24['rating'] = round(coffee_24['rating'], 1)
coffee_24['percent'] = round(coffee_24['qnt']/coffee_24['qnt'].sum()*100, 1)

In [ ]:
coffee_24

Подавляющее количество кофеен (95,8%) не являются круглосуточными, при этом рейтинг у них на 0.1 п.п. выше - 4.3 против 4.2 у круглосуточных. Посмотрим какие часы работы чаще всего бывают у кофеен:

In [ ]:
coffee_data.groupby('hours').agg(qnt = ('name', 'count')).reset_index().sort_values('qnt', ascending = False)

Чаще всего кофейни работают с 10 до 22 часов.

## 3.3. Рейтинги кофеен и их распределение по районам


Рассмотрим детальнее какие в целом у кофеен рейтинги и как они распределяются по районам:

In [ ]:
round(coffee_data['rating'].mean(),1)

In [ ]:
coffee_district_rating = round(coffee_data.groupby('district')['rating'].mean().reset_index()\
                               .sort_values('rating', ascending = False), 1)
coffee_district_rating

Рейтинги кофеен примерно схожи, но в ЮАО, ЮВАО, СВАО и ЗАО они чуть ниже среднего - 4.2 против 4.3 в среднем по рынку. Посмотрим на графике:

In [ ]:
ax = sns.barplot(x="rating", y="district", data=coffee_district_rating)
ax.set_xlim(4, 5)  # устанавливаем ограничение по оси y
ax.set_title('Средний рейтинг кофеен')
ax.set_xlabel('Рейтинг кофеен')
ax.set_ylabel('Административный округ')
plt.show()

## 3.4 Средняя стоимость кружки кофе 

Посмотрим на какую стоимость чашки капучино стоит ориентироваться при открытии:

In [ ]:
coffee_data['middle_coffee_cup'].mean()  # средняя стоимость

In [ ]:
coffee_data['middle_coffee_cup'].median() # медианная

In [ ]:
plt.figure(figsize=(4, 3))
ax = sns.boxplot(y= 'middle_coffee_cup', data=coffee_data)
ax.set_title('Средняя стоимость кружки кофе: ящик с усами')
ax.set_ylabel('Средняя стоимость кофе')
ax.set_ylim(0, 500)
plt.show()

In [ ]:
coffee_data['middle_coffee_cup'].max()

Средняя и медианная стоимость чашки кофе по рынку - 170-173 р., а максимальная - 375.

In [ ]:
coffee_price = (round(coffee_data.groupby('district')['middle_coffee_cup']
                .mean(), 1)
                .reset_index()
                .sort_values('middle_coffee_cup', ascending = False)
               )
coffee_price

Выше всего кружка кофе в среднем стоит в ЗАО, ЦАО и ЮЗАО - от 184 до 190 рублей. На это влияет более высокая аренда помещений в этих районах - в стоимость кружки кофе также закладывается и стоимость аренды помещения, а в ЦАО, ЗАО и ЮЗАО она выше. Плюс выше покупательская способность - клиенты в этих районах чаще готовы заплатить дороже за кофе.

Посмотрим какое соотношение между стоимостью кружки кофе и ее ценой:

In [ ]:
plt.figure(figsize=(8, 3))
coffee_rating_price = round(coffee_data.groupby('rating')['middle_coffee_cup'].mean().reset_index(), 1)
ax = sns.barplot(y="middle_coffee_cup", x="rating", data=coffee_rating_price)
plt.grid(True)
ax.set_title('Средняя стоимость кружки кофе и рейтинг кофеен')
ax.set_xlabel('Рейтинг')
ax.set_ylabel('Средняя стоимость кофе')
plt.xticks(rotation = 45)
plt.show()

 При открытии не стоит превышать средний показатель, чтобы не отпугнуть потенциальных клиентов, а возможно стоит даже рассмотреть возможность сделать ниже рынка - во всяком случае в выбранном районе. Слишком дешевый кофе непопулярен - среди кофеен с рейтингом выше 4 баллов нет кружки кофе ниже 150.

In [ ]:
coffee_rating = (coffee_data
              .groupby('district')
              .agg(qnt=('address', 'nunique'), rating=('rating', 'mean'), \
                   avg_coffee_price=('middle_coffee_cup', 'median'))
              .reset_index().sort_values('qnt', ascending = False)
            )
coffee_rating['rating'] = round(coffee_rating['rating'],1)
coffee_rating

In [ ]:
# финально визуализируем информацию с помощью интерактивного графика:
# используем сокращения названия районов для упрощения визуализации:
coffee_rating['district'] = coffee_rating['district'].replace(
    ['Центральный административный округ', 'Северный административный округ', \
     'Южный административный округ', 'Северо-Восточный административный округ', 'Западный административный округ', \
     'Восточный административный округ', 'Юго-Восточный административный округ', 'Юго-Западный административный округ', \
     'Северо-Западный административный округ'], 
    ['ЦАО', 'САО', 'ЮАО', 'СВАО', 'ЗАО', 'ВАО', 'ЮВАО', 'ЮЗАО', 'СЗАО'])

fig = px.bar(coffee_rating,  x='district', y='qnt', color='avg_coffee_price', \
             title='Распределение кофеен по районам Москвы в разрезе по количеству и средней цене кружки кофе', 
#              text_auto = True
            )
fig.update_xaxes(title_text = 'Район Москвы')
fig.update_yaxes(title_text = '', categoryorder = 'total ascending')  # сортируем данные по убыванию общего кол-ва заведений
fig.show()

## 3.5 Открытие кофейни: выводы и рекомендации

**Выводы:**
<br>
**1.** Всего среди заведений общественного питания Москвы по состоянию на лето 2022 года 1413 кофеен, что составляет 17% от их общего числа. Свыше 30% кофеен расположены в ЦАО, меньше всего кофеен в ЮВАО, ЮЗАО и СЗАО.
<br>
**2.** Большинство кофеен не является круглосуточными, самый популярный график работы - с 10 утра до 10 вечера. 
<br>
**3.** Рейтинги кофеен примерно схожи, но в ЮАО, ЮВАО, СВАО и ЗАО они чуть ниже среднего - 4.2 против 4.3 в среднем по рынку.
<br>
**4.** Средняя и медианная стоимость чашки кофе по рынку - 170-173 р., а максимальная - 375. Выше всего кружка кофе в среднем стоит в ЗАО, ЦАО и ЮЗАО - от 184 до 190 рублей. 
<br>
**5.** При открытии не стоит превышать средний показатель, чтобы не отпугнуть потенциальных клиентов, а возможно стоит даже рассмотреть возможность сделать ниже рынка - во всяком случае в выбранном районе. Слишком дешевый кофе непопулярен - среди кофеен с рейтингом выше 4 баллов нет кружки кофе ниже 150.
<br>
<br>
**Рекомендации:** Предлагаем рассмотреть возможность открыть кофейню в ЮЗАО, где кофеен достаточно мало относительно общего кол-ва кофеен в Москве,при этом средняя цена кружки кофе составляет 198 рублей, что значительно выше средней по Москве. Стоимость кружки кофе при открытии можно установить на уровне 165-170 рублей, что будет ниже средней по Москве и значительно ниже средней по району и позволит привлечь больше клиентов.

# 4. Подготовка презентации

Презентация: https://drive.google.com/file/d/1CoppP2kMJ1PrDft2ZY83JGf7UI22hkKm/view?usp=share_link